# Optimization using Auto-Differentiation

Purpose: To familiarize yourself with the `ad` code, and use it in a simple gradient-based optimization task.

In [1]:
import numpy as np
from ad import *
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Using `Var` and `Operation` objects

In [2]:
X = Var(2.)
A = Var(3.)
F = X**2 * A
print(F)
print(F())

(2.0)**2*3.0
12.0


In [3]:
F.zero_grad()  # reset grads to zero
F.backward()   # propagate derivatives down through graph
print(A.grad)  # dFdA
print(X.grad)  # dFdX

4.0
12.0


In [17]:
A = 4.
print(F)
print(F.evaluate())

(2.0)**2*4
16.0


<div class="alert alert-block alert-info">
    You can even implement your own function (and its derivative) using the template below.
</div>

In [ ]:
def your_function(a):
    '''
     y = a.your_function()
     a is a Var
     y is a Var such that y.val = your_function(a.val)
    '''
    c = YourFunction([a])()
    return c

class YourFunction(Operation):
    def __init__(self, args):
        self.args = args  # store the list of inputs to the function

    def __call__(self):
        # The arguments to the function were saved in self.args.
        # The line below is where you implement your function using NumPy.
        val = self.args[0].val  # <=================
        y = Var(val)       # Create a corresponding Var object
        y.creator = self   # Hook to the operation that create it
        return y

    def __repr__(self):
        return 'your_function('+self.args[0].__repr__()+')'

    def backward(self, s=1.):
        # Implement the derivative (using NumPy), and call
        # backward on the arguments.
        deriv = np.zeros_like(self.args[0].val) # <=========
        self.args[0].backward(s*deriv)


In [ ]:
y = your_function(A)
print(f'{y} =')
print(y())

## Optimize a function

### Build the expression
Let's encode the mathematical expression,
$$
F = 8x^4 + 4x^3 -28 x^2 - 24
$$
and then find its minimum.

In [18]:
# x
X = Var(0.0)

# Coefficients
A = Var(8.0)
B = Var(4.0)
C = Var(-28.0)
D = Var(-24.0)

In [19]:
# Build the formula using the functions from the `ad` module
term1 = A * X**4
term2 = B * X**3
term3 = C * X**2
term4 = D * X
F = term1 + term2 + term3 + term4

In [20]:
# Or you can build it all at once using one of these
#F = A * X**4 + B * X**3 + C * X**2 + D * X
#F = Var(8.)*X**4 + Var(4)*X**3 + Var(-28.)*X**2 + Var(-24)*X

In [21]:
# Display the expression
F

(((8.0*(0.0)**4+4.0*(0.0)**3)+-28.0*(0.0)**2)+-24.0*0.0)

<div class="alert alert-block alert-info">
    Try using the AD code on your own expression.
</div>

### Evaluate the expression

In [22]:
F.val  # for the given X-value as it was set up

0.0

In [23]:
# Can also use functional notation
F()

0.0

In [24]:
# Let's choose a different x-value
X.set(1.)
F.evaluate()  # Causes recomputation of the whole graph

-40.0

In [25]:
F

(((8.0*(1.0)**4+4.0*(1.0)**3)+-28.0*(1.0)**2)+-24.0*1.0)

### Gradients

In [26]:
F.zero_grad()   # sets all gradients to zero (duh)
F.backward()   # projects the derivatives down through the graph
X.grad

-36.0

In [27]:
X.set(-2)
F.evaluate()
F.zero_grad()
F.backward()
print(X.grad)

-120.0


### Gradient-Descent Optimization

In [34]:
# *** YOU MIGHT HAVE TO ADJUST THESE 3 PARAMETERS ***
x = 5    # initial guess at root
kappa = 0.01  # gradient step multiplier
n_iters = 100  # number of gradient-descent steps

# for plotting
xh = []
fh = []

for n in range(n_iters):
    X.set(x)
    F.evaluate()

    # Record values for plotting
    xh.append(X())
    fh.append(F())

    # Compute gradients
    F.zero_grad()
    F.backward()
    
    # Gradient step
    x -= kappa*X.grad
    
    print(X(), F())

5 4680.0
-34.96 11745918.73179648
13472.051947520004 2.635366929354287e+17
-782462083251.4263 2.998770247343866e+48
1.5329919808424804e+35 4.418242197430104e+141


OverflowError: (34, 'Numerical result out of range')

In [ ]:
xx = np.linspace(-2,2,100)  # Choose a bunch of x values
yy = []
# Evaluate expression for each x value
for x in xx:
    X.set(x)
    F.evaluate()
    yy.append(F())
# Plot the graph, and the optimization iterates
plt.plot(xx,yy);
plt.plot(xh, fh, 'r.');